Teste

In [9]:
import random

print ("hello, world!")

hello, world!


In [10]:
%pip install --upgrade pip
%pip install python-constraint

from constraint import *
from itertools import combinations


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip None
error: uninstall-no-record-file

× Cannot uninstall pip None
╰─> The package's contents are unknown: no RECORD file was found for pip.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps pip==25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Instanciar o Problema

In [11]:

days = 'Segunda Terça Quarta Quinta Sexta'.split()
class_groups = 't01 t02 t03'.split()
blocks = 'B01 B02 B03 B04 B05 B06 B07 B08 B09 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B20'.split()
lecturers = 'jo mike rob sue'.split()
courses = 'UC11 UC12 UC13 UC14 UC15 UC21 UC22 UC23 UC24 UC25 UC31 UC32 UC33 UC34 UC35'.split()
rooms = 'A C G Lab01 Lab02'.split()


# Constrains

In [12]:

course_classes = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']
}
class_lecturers = {
    'jo' : ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35']
}
class_rooms = {
    'UC22' : ['Lab01'],
    'UC14' : ['Lab01'],
}

time_restrictions = {
    'mike' : ['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20'],
    'rob' : ['B01', 'B02', 'B03', 'B04'],
    'sue' : ['B09', 'B10', 'B11', 'B12', 'B17', 'B18', 'B19', 'B20']
    }



days_blocks = {
    'Segunda': ['B01', 'B02', 'B03', 'B04'],
    'Terça': ['B05', 'B06', 'B07', 'B08'],
    'Quarta': ['B09', 'B10', 'B11', 'B12'],
    'Quinta': ['B13', 'B14', 'B15', 'B16'],
    'Sexta': ['B17', 'B18', 'B19', 'B20'],
}


online_lessons = {"UC21_2", "UC31_2"}

physical_rooms = [r for r in rooms if r != "Online"]

# Domínios & Funções auxiliares

In [13]:
# Aulas a agendar
lessons_to_schedule = [f"{course_code}_{index}" for course_code in courses for index in (1, 2)]

# Inversões (UC -> docente; UC -> turma)
course_to_teacher = {course_code: lecturer_name for lecturer_name, course_list in class_lecturers.items() for course_code in course_list}
course_to_group = {course_code: group_name for group_name, course_list in course_classes.items() for course_code in course_list}

# ——— Domínio ———
def domain_for_lesson(lesson_name):
    course_code = lesson_name.split("_")[0]
    teacher_name = course_to_teacher[course_code]
    teacher_blocked_blocks = set(time_restrictions.get(teacher_name, []))
    allowed_blocks = [block for block in blocks if block not in teacher_blocked_blocks]

    if lesson_name in online_lessons:
        candidate_rooms = ["Online"]                 # aula online não ocupa sala física
    elif course_code in class_rooms and class_rooms[course_code]:
        candidate_rooms = class_rooms[course_code]   # salas obrigatórias para a UC
    else:
        candidate_rooms = physical_rooms             # qualquer sala física

    return [(block, room) for block in allowed_blocks for room in candidate_rooms]


# ——— Funções para ajudar os Constrains ———

def no_overlapping_blocks(*lesson_assignments):
    # Cada argumento é um par (block, room) de uma variável da restrição
    blocks = [block for (block, room) in lesson_assignments]
    return len(set(blocks)) == len(blocks)

def not_same_physical_room_in_block(lesson_assignment_left, lesson_assignment_right):
    # Uma aula por (sala, bloco), exceto a aula online
    (block_left, room_left) = lesson_assignment_left
    (block_right, room_right) = lesson_assignment_right
    if room_left == "Online" or room_right == "Online":
        return True
    return not (block_left == block_right and room_left == room_right)

block_to_day_map = {block: day for day, day_blocks in days_blocks.items() for block in day_blocks}
def lessons_on_different_days(lesson_assignment_first, lesson_assignment_second):
    # As 2 aulas da mesma UC em dias diferentes
    return block_to_day_map[lesson_assignment_first[0]] != block_to_day_map[lesson_assignment_second[0]]

def lessons_four_days(*lesson_assignments):
    # Uma turma deve ter aulas em 4 dias exatamente
    days_with_classes = {block_to_day_map[block] for (block, room) in lesson_assignments}
    return len(days_with_classes) == 4

def make_max_three(blocks):
    # Máximo de 3 aulas por dia por turma
    blocks = tuple(blocks)
    def _rule(*lesson_assignments):
        return sum(1 for (block, room) in lesson_assignments if block in blocks) <= 3
    return _rule

def all_classes_on_day_are_consecutive(*lesson_assignments):
    # Se uma turma tem aulas num dia, essas aulas devem ser em blocos consecutivos.
    day_to_blocks = {}
    for block, room in lesson_assignments:
        day = block_to_day_map[block]
        day_to_blocks.setdefault(day, []).append(int(block[1:]))

    for day, block_numbers in day_to_blocks.items():
        if len(block_numbers) > 1:
            block_numbers.sort()
            # Verifica se os números dos blocos formam uma sequência contínua
            # [5, 6, 7] -> max(7) - min(5) = 2, que é len(3) - 1. Válido.
            # [5, 7] -> max(7) - min(5) = 2, que é != len(2) - 1. Inválido.
            if max(block_numbers) - min(block_numbers) != len(block_numbers) - 1:
                return False
    return True



# Hard Constrains

In [14]:

def addHardConstrains(problem):
    # 4a) Por docente
    for lecturer_name in lecturers:
        variables_for_lecturer = [f"{course_code}_{index}" for course_code in class_lecturers[lecturer_name] for index in (1, 2)]
        problem.addConstraint(no_overlapping_blocks, tuple(variables_for_lecturer))

    # 4b) Por turma
    for group_name in class_groups:
        variables_for_group = [f"{course_code}_{index}" for course_code in course_classes[group_name] for index in (1, 2)]
        problem.addConstraint(no_overlapping_blocks, tuple(variables_for_group))

    # 4c) Por sala
    for left_lesson, right_lesson in combinations(lessons_to_schedule, 2):
        problem.addConstraint(not_same_physical_room_in_block, (left_lesson, right_lesson))

    #4f) Max 3 aulas por dia por turma
    for group_name in class_groups:
        vars_for_group = [
            lesson for lesson in lessons_to_schedule
            if course_to_group[lesson.split("_")[0]] == group_name
        ]
        for day, day_blocks in days_blocks.items():
            problem.addConstraint(make_max_three(day_blocks), tuple(vars_for_group))



# Soft Constrains

In [15]:
def evaluateSolution(solution):
    penalty = 0
    for group_name in class_groups:
        group_lessons = [
            lesson for lesson in lessons_to_schedule
            if course_to_group[lesson.split("_")[0]] == group_name
        ]
        assignments = [solution[lesson] for lesson in group_lessons]
        if not all_classes_on_day_are_consecutive(*assignments):
            penalty += 10
        if not lessons_four_days(*assignments):
            penalty += 100
    for course_code in courses:
        first = solution[f"{course_code}_1"]
        second = solution[f"{course_code}_2"]
        if not lessons_on_different_days(first, second):
            penalty += 50
    return penalty

# Descobrir o resultado

In [16]:
bestSolution = None
iterations = 1000
minScore = float("inf")

lesson_domains = {lesson: domain_for_lesson(lesson) for lesson in lessons_to_schedule}
ordered_lessons = sorted(lessons_to_schedule, key=lambda lesson: len(lesson_domains[lesson]))

for iteration in range(iterations):
    solver = MinConflictsSolver()
    solver.random = random.Random(iteration)
    tt_problem = Problem(solver)
    for lesson in ordered_lessons:
        tt_problem.addVariable(lesson, lesson_domains[lesson])
    addHardConstrains(tt_problem)
    solution = tt_problem.getSolution()
    if not solution:
        continue

    penalty = evaluateSolution(solution)
    if penalty < minScore:
        minScore = penalty
        bestSolution = solution
        print(f"Iteration {iteration + 1} of {iterations}, new best penalty: {minScore}")
    elif (iteration + 1) % 200 == 0:
        if minScore == float("inf"):
            print(f"Iteration {iteration + 1} of {iterations}, no feasible solution yet.")
        else:
            print(f"Iteration {iteration + 1} of {iterations}, best penalty so far: {minScore}")

    if minScore == 0:
        break

print(f"\nMelhor solução com penalidade {minScore}:\n")

if not bestSolution:
    print("Sem solução encontrada.")
else:
    # agenda dia//bloco
    schedule = {day: {block: [] for block in days_blocks[day]} for day in days}
    for lesson, (block, room) in bestSolution.items():
        course = lesson.split("_")[0]
        entry = {
            "lesson": lesson,
            "course": course,
            "group": course_to_group[course],
            "teacher": course_to_teacher[course],
            "room": room,
            "block": block,
            "day": block_to_day_map[block],
        }
        schedule[entry["day"]][block].append(entry)

    # ordenar dia//bloco
    for day in days:
        print(f"\n=== {day} ===")
        for block in days_blocks[day]:
            entries = schedule[day][block]
            if not entries:
                continue
            print(f"{block}:")
            # ordena por sala e curso para consistência visual
            for e in sorted(entries, key=lambda x: (x["room"], x["course"])):
                print(f"  {e['room']}: {e['course']} ({e['group']}, {e['teacher']})")

    # lista ordenada por bloco (B01..B20)
    print("\n--- Lista por bloco ---")
    ordered = sorted(
        [
            {
                "lesson": lesson,
                "course": lesson.split("_")[0],
                "group": course_to_group[lesson.split("_")[0]],
                "teacher": course_to_teacher[lesson.split("_")[0]],
                "room": room,
                "block": block,
                "day": block_to_day_map[block],
            }
            for lesson, (block, room) in bestSolution.items()
        ],
        key=lambda e: int(e["block"][1:])
    )
    for e in ordered:
        print(f"{e['block']} ({e['day']}): {e['room']} - {e['course']} ({e['group']}, {e['teacher']})")

Iteration 1 of 1000, new best penalty: 510
Iteration 2 of 1000, new best penalty: 420
Iteration 7 of 1000, new best penalty: 280
Iteration 42 of 1000, new best penalty: 270


KeyboardInterrupt: 